# Construcción y Uso de Una Curva Cero Cupón

Se importa la versión de `QC_Financial` compilada para Python3.

In [1]:
from finrisk import QC_Financial_3 as Qcf

Librerías adicionales.

In [3]:
import pandas as pd

Para formateo de `pandas.DataFrames`.

In [4]:
format_dict = {'nominal': '{0:,.2f}', 'amort': '{0:,.2f}', 'interes': '{0:,.2f}', 'flujo': '{0:,.2f}',
               'icp_inicial': '{0:,.2f}', 'icp_final': '{0:,.2f}',
               'valor_tasa': '{0:,.4%}', 'spread': '{0:,.4%}', 'gearing': '{0:,.2f}',
               'amort_moneda_pago': '{0:,.2f}', 'interes_moneda_pago': '{0:,.2f}', 'valor_indice_fx': '{0:,.2f}'}

## Construcción de la Curva

La construcción de una curva se hace en varios pasos.

### Vectores de Float e Int

In [5]:
# Este es un vector de números enteros (grandes, de ahí la l (long))
lvec = Qcf.long_vec()

In [6]:
# Agregar un elemento
lvec.append(1)

In [7]:
# Este es un vector de números double.
vec = Qcf.double_vec()

In [8]:
# Agregar un elemento
vec.append(.025)

In [9]:
# Obtener ese elemento
print("Tasa: {0:,.2%}".format(vec[0]))

Tasa: 2.50%


### Objeto Curva

Es simplemente un `long_vec` que representa las abscisas de la curva y un `double_vec` que representa las ordenadas. Ambos vectores deben tener el mismo largo. 

In [10]:
zcc = Qcf.QCCurve(lvec, vec)

Un elemento de una curva se representa como un par abscisa, ordenada.

In [11]:
zcc.get_values_at(0)

Se obtiene el plazo en una posición de la curva.

In [12]:
zcc.get_values_at(0).tenor

1

Se obtiene la tasa en una posición de la curva.

In [13]:
zcc.get_values_at(0).value

0.025

Se agrega un par (plazo, valor) a la curva.

In [14]:
zcc.set_pair(30, .026)

Se verifica.

In [15]:
# Plazo
zcc.get_values_at(1).tenor

30

In [16]:
# Valor
zcc.get_values_at(1).value

0.026

Se agrega un par más.

In [17]:
zcc.set_pair(370, .03)

Se itera sobre la curva mostrando sus valores

In [18]:
for i in range(0, zcc.get_length()):
    pair = zcc.get_values_at(i)
    print("Tenor: {0:} Valor: {1:.4%}".format(pair.tenor, pair.value))

Tenor: 1 Valor: 2.5000%
Tenor: 30 Valor: 2.6000%
Tenor: 370 Valor: 3.0000%


Se define un interpolador. En este caso, un interpolador lineal.

In [19]:
lin = Qcf.QCLinearInterpolator(zcc)

Se hace una prueba.

In [20]:
print("Tasa en {0:} es igual a {1:.4%}".format(10, lin.interpolate_at(10)))

Tasa en 10 es igual a 2.5310%


Para completar el proceso se define una fracción de año, un factor de capitalización y un tipo de tasa. Con estos objetos se termina de dar de alta una curva cero.

In [21]:
yf = Qcf.QCAct365()
wf = Qcf.QCCompoundWf()
tasa = Qcf.QCInterestRate(.01, yf, wf)

In [22]:
zz = Qcf.ZeroCouponCurve(lin, tasa)

El interpolador permite obtener una tasa a cualquier plazo.

In [23]:
plazo = 365
print("Tasa en {0:} es igual a {1:.4%}".format(plazo, zz.get_rate_at(plazo)))

Tasa en 365 es igual a 2.9941%


In [24]:
type(zz)

finrisk.QC_Financial_3.ZeroCouponCurve

TODO: mostrar los otros métodos disponibles.

In [25]:
zz.get_discount_factor_at(1)

0.9999323512948787

## Valorizar

Se da de alta un objeto `PresentValue`.

In [26]:
pv = Qcf.PresentValue()

### Depósito a Plazo

Se utilizará como instrumento un depósito a plazo en CLP o USD. Este instrumento se modela como un `SimpleCashflow`. Este, a su vez se construye con un monto, una fecha y una moneda.

In [27]:
# Con estas variables vamos a construir
fecha_vcto = Qcf.QCDate(12, 1, 2020)
monto = 10000000.0
clp = Qcf.QCCLP()

# Se construye el depósito
depo = Qcf.SimpleCashflow(fecha_vcto, monto, clp)

In [28]:
print("Monto del depósito: {0:,.0f}".format(depo.amount()))

Monto del depósito: 10,000,000


Se define una fecha de valorización y se calcula el valor presente del depo.

In [29]:
fecha_hoy = Qcf.QCDate(30, 8, 2019)
print("Valor presente depo: {0:,.0f}".format(pv.pv(fecha_hoy, depo, zz)))

Valor presente depo: 9,901,106


Se verifica *a mano* el resultado.

In [30]:
plazo = fecha_hoy.day_diff(fecha_vcto)
print("Plazo:", plazo)

Plazo: 135


In [31]:
tasa_int = zz.get_rate_at(plazo)
print("Tasa: {0:,.4%}".format(tasa_int))

Tasa: 2.7235%


In [32]:
valor_presente = monto * (1 + tasa_int)**(-plazo / 365)
print("Valor presente a mano: {0:,.0f}".format(valor_presente))

Valor presente a mano: 9,901,106


### Fixed Rate Leg

Construyamos una curva un poco más real.

In [33]:
curva = pd.read_excel("20190823_curva_camara_clp.xlsx")
curva.style.format({"tasa":"{0:,.4%}"})

FileNotFoundError: [Errno 2] No such file or directory: '20190823_curva_camara_clp.xlsx'

In [ ]:
lvec1 = Qcf.long_vec()
vec1 = Qcf.double_vec()
for index, row in curva.iterrows():
    lvec1.append(int(row['plazo']))
    vec1.append(row['tasa'])

In [ ]:
zcc1 = Qcf.QCCurve(lvec1, vec1)
lin1 = Qcf.QCLinearInterpolator(zcc1)
zz1 = Qcf.ZeroCouponCurve(lin1, tasa)

Se da de alta una pata fija a 5Y:

In [ ]:
# Se da de alta los parámetros requeridos
rp = Qcf.RecPay.RECEIVE
fecha_inicio = Qcf.QCDate(27, 8, 2019)
fecha_final = Qcf.QCDate(27, 8, 2024) 
bus_adj_rule = Qcf.BusyAdjRules.MODFOLLOW
periodicidad = Qcf.Tenor('6M')
periodo_irregular = Qcf.StubPeriod.SHORTFRONT
calendario = Qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
nominal = 100000000.0
amort_es_flujo = True
valor_tasa_fija = .0209
tasa_cupon = Qcf.QCInterestRate(valor_tasa_fija, Qcf.QCAct360(), Qcf.QCLinearWf())
moneda = Qcf.QCCLP()
es_bono = False

# Se da de alta el objeto
fixed_rate_leg = Qcf.LegFactory.build_bullet_fixed_rate_leg(rp,
                                                            fecha_inicio,
                                                            fecha_final,
                                                            bus_adj_rule,
                                                            periodicidad,
                                                            periodo_irregular,
                                                            calendario,
                                                            lag_pago,
                                                            nominal,
                                                            amort_es_flujo,
                                                            tasa_cupon,
                                                            moneda,
                                                            es_bono)

In [ ]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, fixed_rate_leg.size()):
    tabla.append(Qcf.show(fixed_rate_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_ini', 'fecha_fin', 'fecha_pago', 'nominal', 'amort', 'interes', 'amort_es_flujo', 'flujo', 'moneda',
            'valor_tasa', 'tipo_tasa']
df = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df.style.format(format_dict)

Se calcula ahora el valor presente:

In [ ]:
vp_fija = pv.pv(fecha_hoy, fixed_rate_leg, zz1)
print("Valor presente de la pata fija es: {0:,.0f}".format(vp_fija))